In [11]:
## Imports 

import numpy as np    ## Math 
import requests as rq ## API get requests
import pandas as pd   ## Dataframes 
import datetime       ## Track time 
import time           ## Wait times for API

In [12]:
## Get binance info
binance_info = rq.get("https://api.binance.com/api/v1/exchangeInfo").json()
binance_info_symbols = binance_info["symbols"]
binance_symbols = []

## Construct list of total markets 
for each_dict in binance_info_symbols:
    binance_symbols.append(each_dict["symbol"])

## Construct list of alt markets
binance_keep = []
for each_market in binance_symbols:
    if each_market[-3:] == "ETH":
        binance_keep.append(each_market[0:(len(each_market) - 3)])
    
## Get bittrex info
bittrex_info = rq.get("https://bittrex.com/api/v1.1/public/getmarketsummaries").json()
bittrex_info_symbols = bittrex_info["result"]
bittrex_symbols = []

## Construct list of total markets 
for each_dict in bittrex_info_symbols:
    bittrex_symbols.append(each_dict["MarketName"])
    
## Construct list of alt markets 
bittrex_keep = []
for each_market in bittrex_symbols:
    if each_market[0:3] == "ETH":
        bittrex_keep.append(each_market[each_market.rindex('-')+1:])
        
## Get intersection 
keep = set(binance_keep).intersection(bittrex_keep)

## Now make lists of pairs we can send to the apis 
symbols_binance = []
for alt in keep: 
    symbols_binance.append(alt + "ETH")
    
symbols_bittrex = []
for alt in keep:
    symbols_bittrex.append("ETH-" + alt)

In [3]:
print(keep)

{'ADA', 'DNT', 'WAVES', 'XMR', 'SNT', 'STRAT', 'WINGS', 'MANA', 'ZEC', 'XRP', 'LUN', 'ENG', 'POWR', 'BNT', 'NEO', 'OMG', 'BAT', 'SALT', 'STORJ', 'ETC', 'RLC', 'FUN', 'DASH', 'ADX', 'RCN', 'VIB', 'XLM', 'BCC', 'BTG', 'MCO', 'QTUM', 'LTC'}


In [19]:
def get_data(binance_symbol = "XRPETH",
             bittrex_symbol = "ETH-ERP",
             binance_limit  = 50
            ):
    
    ## Get market info from binance 
    payload = {'symbol': binance_symbol, "limit": binance_limit}
    binance = rq.get("https://api.binance.com/api/v1/depth", params = payload).json()

    ## Get time
    binance_time = datetime.datetime.now()

    ## Get market info from bittrex 
    bittrex = rq.get("https://bittrex.com/api/v1.1/public/getorderbook?market=" + bittrex_symbol + "&type=both").json()

    ## Get time
    bittrex_time = datetime.datetime.now()
    
    return [binance, bittrex, binance_time, bittrex_time, binance_symbol]

In [431]:
crypto_data = get_data(binance_symbol = "XRPETH",
             bittrex_symbol = "ETH-XRP",
             binance_limit  = 50,
             )

In [14]:
def binance_to_array(data): 

    ## Make 2-D numpy array with price and quantity

    binance_array = np.empty( (0,2) , dtype = "float64")

    for each_list in data:
        binance_array = np.append(binance_array, np.array([[each_list[0], each_list[1]]], dtype = "float64"), 
                                  axis = 0)
        
    return binance_array

In [15]:
def bittrex_to_array(data):

    ## Similarly, make a 2-D numpy array 

    bittrex_array = np.empty( (0,2) )

    for each_dict in data:
        bittrex_array = np.append(bittrex_array, np.array([[each_dict["Rate"], each_dict["Quantity"]]], dtype = "float64"), 
                                      axis = 0)
        
    return bittrex_array
    

In [16]:
def clean_data(data):
    
    ## Initial cleaning/subsetting
    binance_bids = data[0]["bids"]
    binance_asks = data[0]["asks"]
    
    bittrex_bids = data[1]["result"]["buy"]
    bittrex_asks = data[1]["result"]["sell"]
    
    ## Create numpy arrays with relevent information 
    binance_bids_array = binance_to_array(data = binance_bids)
    binance_asks_array = binance_to_array(data = binance_asks)

    bittrex_bids_array = bittrex_to_array(data = bittrex_bids)
    bittrex_asks_array = bittrex_to_array(data = bittrex_asks)
    
    ## FOR DEBUGGING 
    print(len(binance_bids_array))
    print(len(binance_asks_array))
    print(len(bittrex_bids_array))
    print(len(bittrex_asks_array))
    
    ## Find smallest length
    lengths = [len(binance_bids_array), len(binance_asks_array), len(bittrex_bids_array), len(bittrex_asks_array)]
    min_length = min(lengths)
    
    ## Construct pandas dataframe 
    full_data = pd.DataFrame({"binance_bids_price":binance_bids_array[:, 0][0:min_length],
                              "binance_bids_quantity":binance_bids_array[:, 1][0:min_length],
                              "binance_asks_price":binance_asks_array[:, 0][0:min_length],
                              "binance_asks_quantity":binance_asks_array[:, 1][0:min_length],
                              "bittrex_bids_price":bittrex_bids_array[:, 0][0:min_length],
                              "bittrex_bids_quantity":bittrex_bids_array[:, 1][0:min_length],
                              "bittrex_asks_price":bittrex_asks_array[:, 0][0:min_length],
                              "bittrex_asks_quantity":bittrex_asks_array[:, 1][0:min_length]})

    ## Add price difference variables 
    full_data["binance_difference"] = full_data["binance_bids_price"] - full_data["bittrex_asks_price"]
    full_data["bittrex_difference"] = full_data["bittrex_bids_price"] - full_data["binance_asks_price"]
    full_data["binance_time"] = data[2]
    full_data["bittrex_time"] = data[3]
    full_data["market"] = data[4]
    
    return [full_data, data[4]]


In [436]:
cleaned_crypto_data = clean_data(data = crypto_data)

In [438]:
cleaned_crypto_data[0].head()

,binance_asks_price,binance_asks_quantity,binance_bids_price,binance_bids_quantity,bittrex_asks_price,bittrex_asks_quantity,bittrex_bids_price,bittrex_bids_quantity,binance_difference,bittrex_difference,binance_time,bittrex_time
0,0.001393,1907.0,0.001391,118.0,0.001393,230.300173,0.001381,0.458779,-0.000002,-0.000012,2018-01-14 13:50:24.724089,2018-01-14 13:50:24.814334
1,0.001394,1838.0,0.001391,373.0,0.001395,1010.000000,0.001379,36.137892,-0.000004,-0.000015,2018-01-14 13:50:24.724089,2018-01-14 13:50:24.814334
2,0.001395,444.0,0.001390,71.0,0.001396,15.225558,0.001379,101.817690,-0.000006,-0.000016,2018-01-14 13:50:24.724089,2018-01-14 13:50:24.814334
3,0.001395,787.0,0.001390,218.0,0.001400,14.540555,0.001378,14.732928,-0.000010,-0.000017,2018-01-14 13:50:24.724089,2018-01-14 13:50:24.814334
4,0.001395,64.0,0.001390,1405.0,0.001400,1677.191489,0.001376,25.320162,-0.000010,-0.000019,2018-01-14 13:50:24.724089,2018-01-14 13:50:24.814334


In [20]:
def write_csvfiles(cleaned_data):
    
    ## Get timestamp for data in correct format 
    date = cleaned_data[0]["binance_time"][0]
    string_date = date.strftime("%Y-%m-%d_%H:%M:%S.%f")

    full_file = "/Users/rkmoss/Desktop/crypto/data/" + cleaned_data[1] + "_" + string_date + ".csv"
    
    ## Write csv file containing relevant information
    cleaned_data[0].to_csv(full_file)
    
    print(full_file)

In [442]:
write_csvfiles(cleaned_data = cleaned_crypto_data)

In [23]:
def workflow(binance_symbols,
             bittrex_symbols,
             wait_time_s = 10,
             total_time_m = 10):
    
    ## Full data gathering process
    
    ## Keep track of time
    starting_time = datetime.datetime.now()
    
    ## while time elapsed is less than total time in minutes
    while((datetime.datetime.now() - starting_time).seconds < (total_time_m * 60)):
        
        ## Loop through the available symbol pairs and gather data 
        for this_binance_symbol, this_bittrex_symbol in zip(binance_symbols, bittrex_symbols):
            
            ## FOR DEBUGGING
            print(this_binance_symbol)
            print(this_bittrex_symbol)
            
            ## Get data for this symbol
            crypto_data = get_data(binance_symbol = this_binance_symbol,
                                   bittrex_symbol = this_bittrex_symbol,
                                   binance_limit  = 50
                                   )
            
            ## Clean this data
            cleaned_crypto_data = clean_data(data = crypto_data)
            
            ## Write csv files for this data including the relevant information
            write_csvfiles(cleaned_data = cleaned_crypto_data)
            
            ## Wait some amount of time to prevent spamming the API
            time.sleep(wait_time_s)

In [24]:
workflow(binance_symbols = symbols_binance,
         bittrex_symbols = symbols_bittrex,
         wait_time_s = 10,
         total_time_m = 10)

ADAETH
ETH-ADA
50
50
100
100
/Users/rkmoss/Desktop/crypto/data/ADAETH_2018-01-14_15:22:07.204436.csv
DNTETH
ETH-DNT
50
50
62
100
/Users/rkmoss/Desktop/crypto/data/DNTETH_2018-01-14_15:22:18.015104.csv
WAVESETH
ETH-WAVES
50
50
43
100
/Users/rkmoss/Desktop/crypto/data/WAVESETH_2018-01-14_15:22:28.886603.csv
XMRETH
ETH-XMR
50
50
94
100
/Users/rkmoss/Desktop/crypto/data/XMRETH_2018-01-14_15:22:39.979860.csv
SNTETH
ETH-SNT
50
50
100
100
/Users/rkmoss/Desktop/crypto/data/SNTETH_2018-01-14_15:22:50.808046.csv
STRATETH
ETH-STRAT
50
50
73
100
/Users/rkmoss/Desktop/crypto/data/STRATETH_2018-01-14_15:23:01.696945.csv
WINGSETH
ETH-WINGS
50
50
42
100
/Users/rkmoss/Desktop/crypto/data/WINGSETH_2018-01-14_15:23:13.099317.csv
MANAETH
ETH-MANA
50
50
49
100
/Users/rkmoss/Desktop/crypto/data/MANAETH_2018-01-14_15:23:24.032248.csv
ZECETH
ETH-ZEC
50
50
60
100
/Users/rkmoss/Desktop/crypto/data/ZECETH_2018-01-14_15:23:34.997295.csv
XRPETH
ETH-XRP
50
50
100
100
/Users/rkmoss/Desktop/crypto/data/XRPETH_2018-01